# 浜名湖4月8日 9R 結果

In [9]:
sample_html = \
    'http://boatrace.jp/owpc/pc/race/raceresult?rno=9&jcd=06&hd=20200410'

In [10]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

## htmlをsoupに取り込む

In [11]:
target_url = sample_html
__html_content = urlopen(target_url).read()
__soup = BeautifulSoup(__html_content, 'html.parser')

### 結果表を抜く

In [7]:
__target_table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div.grid.is-type2.h-clear.h-mt10 > '\
    'div:nth-child(1) > div > table'
__target_table_html = __soup.select_one(__target_table_selector)
player_res_html_list = __target_table_html.select('tbody')

In [46]:
waku_dict = {}
for rank_p_html in player_res_html_list:
    rank, waku, name, time = \
        list(map(lambda x: x.text, rank_p_html.select('td')))
    # rankはF,L欠などが存在するためエラーハンドルがいる
    try:
        rank = int(rank)
    except ValueError:
        rank = -1
#     time = time.replace('"', '.')
    waku = int(waku)
    name = name.replace('\n', '').replace('\u3000', '').replace(' ', '')
    no, name = name.split('\r')
    no = int(no)

    __content_dict = {
        'rank': rank,
        'name': name,
        'time': time
    }

    waku_dict[waku] = __content_dict

### 結果STテーブルを抜く

In [6]:
from getdata import CommonMethods4Official
cmo = CommonMethods4Official()
__target_table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > '\
    'div.grid.is-type2.h-clear.h-mt10 > '\
    'div:nth-child(2) > div > table'
st_list = cmo.getSTtable2tuple(__soup, __target_table_selector, 1)

In [7]:
goutei_list = list(
            map(lambda x: int(x.select('div > span')[0].text),
                st_list))

In [8]:
goutei_list

[1, 3, 6, 2, 4, 5]

In [30]:
__tenji_C_idx = goutei_list.index(6)
tenji_C = __tenji_C_idx + 1

In [31]:
# 展示ST抜き出し
tenji_st_time_list = list(
    map(lambda x: x.select('div > span')[2].text,
        st_list))

In [32]:
tenji_ST = __tenji_st_time_list[__tenji_C_idx]

In [33]:
tenji_ST = cmo.rmletter2float(tenji_ST.replace('F', '-'))

### 水面気象情報

In [12]:
from getdata import CommonMethods4Official
cmo = CommonMethods4Official()
table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div:nth-child(5) > '\
    'div:nth-child(2) > div.grid.is-type6.h-clear > '\
    'div:nth-child(1) > div > div.weather1_body.is-type1__3rdadd'
__target_table_html = __soup.select_one(table_selector)
condinfo_html_list = __target_table_html.select('div')
assert len(condinfo_html_list) == 12, \
    f"lengh is not 12:{len(condinfo_html_list)}"
# 気温は2番目のdiv
__tmp_info_html = condinfo_html_list[1]
# spanで情報がとれる (1番目： '気温', 2番目: 数字℃)
__tmp_info = __tmp_info_html.select('span')
temp = __tmp_info[1].text
temp = cmo.rmletter2float(temp)
# 天気は2番目のdiv
__weather_info_html = condinfo_html_list[2]
# spanのなか（1個しかない)
weather = __weather_info_html.select_one('span').text
weather = weather.replace('\n', '')\
                 .replace('\r', '')\
                 .replace(' ','')
# 風速は5番目のdiv
wind_v = cmo.choose_2nd_span(condinfo_html_list[4])
wind_v: int = cmo.rmletter2int(wind_v)

# 水温は8番目のdiv
w_temp = cmo.choose_2nd_span(condinfo_html_list[7])
w_temp = cmo.rmletter2float(w_temp)

# 波高は10番目のdiv
wave = cmo.choose_2nd_span(condinfo_html_list[9])
wave = cmo.rmletter2int(wave)

# 風向きは7番目のdiv
# 画像のみの情報なので，16方位の数字（画像の名前）を抜く
# p中のクラス名2番目にある
wind_dr = condinfo_html_list[6].select_one('p')['class'][1]
wind_dr = cmo.rmletter2int(wind_dr)

content_dict = {
    'temp': temp,
    'weather': weather,
    'wind_v': wind_v,
    'w_temp': w_temp,
    'wave': wave,
    'wind_dr': wind_dr
}

In [13]:
content_dict

{'temp': 16.0,
 'weather': '晴',
 'wind_v': 6,
 'w_temp': 16.0,
 'wave': 4,
 'wind_dr': 3}